This is a simple illustration of a [Particle Swarm](https://en.wikipedia.org/wiki/Particle_swarm_optimization) trying to find the minimum of the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function). Obviously the code is imperfect, but this should serve as a useful illustration.

Parameters can be set in the "PSO" function in the last cell of this notebook. Consider the following tasks:
- Higher search space dimensions (you may need more time in this case(
- change the population size (does a larger swarm imply shorter search time?>)
- Try out other parameter values
    - increase or decreast any of w, a1 or a2
    - change sign of w
- Implement a different goal function, e.g. the [Rastrigin function](https://en.wikipedia.org/wiki/Rastrigin_function). 
- Add an avarage for statistical evaluation (like wedid in the GA notebook)
- If you know python well, you can try to include an animation, e.g. to visualise search biasses 

In [2]:
import numpy as np
import itertools

The following will be the goal ("fitness") function. Here it is to be minimised.

In [3]:
def rosenbrock(pos,dim):    #this serves as a goal function
                            # Defined by f(x,y) = (a-x)^2 + b(y-x^2)^2
                            # Using here: a = 1, b= 100, optimum 0 at (1,1)
        if dim==2:
            return ((1-pos[0])**2 + 100*(pos[1] - pos[0]**2)**2)
        elif dim==1:
            return (1-pos[0])**2 
        else: 
            ros=0;
            for i in range(dim-1):
                ros=ros+100*(pos[i+1] - pos[i]**2)**2 * (1-pos[i])**2 
            return ros

In [4]:
class Particle: # all the material that is relavant at the level of the individual particles
    
    def __init__(self, dim, minx, maxx):
        self.position = np.random.uniform(low=minx, high=maxx, size=dim)
        self.velocity = np.random.uniform(low=-0.1, high=0.1, size=dim)
        self.best_particle_pos = self.position
        self.dim = dim

        self.fitness = rosenbrock(self.position,dim)
        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos):
        self.position = pos
        self.fitness = rosenbrock(self.position,self.dim)
        if self.fitness<self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos

    def updateVel(self, inertia, a1, a2, best_self_pos, best_swarm_pos):
                # Here we use the canonical version
                # V <- inertia*V + a1r1 (peronal_best - current_pos) + a2r2 (global_best - current_pos)
        cur_vel = self.velocity
        r1 = np.random.uniform(low=0, high=1, size = self.dim)
        r2 = np.random.uniform(low=0, high=1, size = self.dim)
        a1r1 = np.multiply(a1, r1)
        a2r2 = np.multiply(a2, r2)
        best_self_dif = np.subtract(best_self_pos, self.position)
        best_swarm_dif = np.subtract(best_swarm_pos, self.position)
                    # the next line is the main equation, namely the velocity update, 
                    # the velocities are added to the positions at swarm level 
        return inertia*cur_vel + np.multiply(a1r1, best_self_dif) + np.multiply(a2r2, best_swarm_dif)


In [5]:
class PSO: # all the material that is relavant at swarm leveel

    def __init__(self, w, a1, a2, dim, population_size, time_steps, search_range):

        # Here we use values that are (somewhat) known to be good
        # There are no "best" parameters (No Free Lunch), so try using different ones
        # There are several papers online which discuss various different tunings of a1 and a2
        # for different types of problems
        self.w = w # Inertia
        self.a1 = a2 # Attraction to personal best
        self.a2 = a2 # Attraction to global best
        self.dim = dim

        self.swarm = [Particle(dim,-search_range,search_range) for i in range(population_size)]
        self.time_steps = time_steps
        print('init')

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = np.random.uniform(low=-500, high=500, size=dim)
        self.best_swarm_fitness = 1e100

    def run(self):
        for t in range(self.time_steps):
            for p in range(len(self.swarm)):
                particle = self.swarm[p]

                new_position = particle.position + particle.updateVel(self.w, self.a1, self.a2, particle.best_particle_pos, self.best_swarm_pos)
                                
                if new_position@new_position > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                    print('Time:', t,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
                    raise SystemExit('Most likely divergent: Decrease parameter values')
 
                self.swarm[p].setPos(new_position)

                new_fitness = rosenbrock(new_position,self.dim)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position

            if t % 100 == 0: #we print only two components even it search space is high-dimensional
                print("Time: %6d,  Best Fitness: %14.6f,  Best Pos: %9.4f,%9.4f" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ")
                if self.dim>2: 
                    print('...')
                else:
                    print('')

Standard values are dim=2, w=0.7, a1=2.02, a2=2.02, population_size=30; but feel free to try others ones.

In [13]:
PSO(dim=2, w=0.7, a1=2, a2=2, population_size=30, time_steps=1001, search_range=100).run()

init
Time:      0,  Best Fitness:   48644.168626,  Best Pos:   -8.1431,  88.3465 
Time:    100,  Best Fitness:       0.005527,  Best Pos:    0.9328,   0.8733 
Time:    200,  Best Fitness:       0.000081,  Best Pos:    1.0011,   1.0032 
Time:    300,  Best Fitness:       0.000081,  Best Pos:    1.0011,   1.0032 
Time:    400,  Best Fitness:       0.000011,  Best Pos:    1.0021,   1.0039 
Time:    500,  Best Fitness:       0.000011,  Best Pos:    1.0021,   1.0039 
Time:    600,  Best Fitness:       0.000011,  Best Pos:    1.0021,   1.0039 
Time:    700,  Best Fitness:       0.000011,  Best Pos:    1.0021,   1.0039 
Time:    800,  Best Fitness:       0.000011,  Best Pos:    1.0021,   1.0039 
Time:    900,  Best Fitness:       0.000011,  Best Pos:    1.0021,   1.0039 
Time:   1000,  Best Fitness:       0.000007,  Best Pos:    0.9979,   0.9959 
